In [1]:
%%writefile requirements.txt
tensorflow==2.3.1
nltk==3.5
colorama==0.4.3
numpy==1.18.5
scikit_learn==0.23.2
Flask==1.1.2
mlflow

Overwriting requirements.txt


In [2]:
!pip3 install -r requirements.txt

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
import json 
import os
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import mlflow

In [4]:
#Mlflow settings
MLFLOW_URL=os.getenv('MLFLOW_URL')
EXPERIMENT=os.getenv('EXPERIMENT')
EPOCHS=int(os.getenv('EPOCHS'))
VERSION=os.getenv('VERSION')
LOSS=os.getenv('LOSS')
OPTIMIZER=os.getenv('OPTIMIZER')

#Set tags
tags={}
tags['TYPE']='IMAGE_RECOGNITION'
tags['TRANSFER_LEARNING']='NO'
tags['TFHUB']='NO'

SAVE_PATH='saved_model'
%mkdir -p saved_model

#Mlflow settings

#set MLflow server 
mlflow.set_tracking_uri(MLFLOW_URL)

#Set experiment
!mlflow experiments create --experiment-name $EXPERIMENT --artifact-location s3://mlflowruns/
exp_id = mlflow.set_experiment(EXPERIMENT)
    
#Close active runs
if mlflow.active_run():
    mlflow.end_run()

Traceback (most recent call last):
  File "/opt/conda/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.7/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mlflow/experiments.py", line 45, in create
    exp_id = store.create_expe

INFO: 'npl/chatbot' does not exist. Creating a new experiment


In [5]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [6]:
print("training sentences: ", len(training_sentences)) #holds all the training data (which are the sample messages in each intent category)
print("training labels: ", len(training_labels)) #holds all the target labels correspond to each training data
print("labels: ", len(labels))
print("responses: ", len(responses))

training sentences:  33
training labels:  33
labels:  8
responses:  8


In [7]:
#convert the target labels into a model understandable form
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [8]:
#vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number.
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len) #used to make all the training text sequences into the same size

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=LOSS, 
              optimizer=OPTIMIZER, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [10]:
from mlflow import tensorflow
mlflow.tensorflow.autolog()

In [11]:
with mlflow.start_run(run_id=None, experiment_id=exp_id, run_name=None, nested=False):
  
  history = model.fit(padded_sequences, np.array(training_labels), epochs=EPOCHS)

  hist=history.history

  #mlflow autolog
  #mlflow.tensorflow.autolog()

  #Set tags
  mlflow.set_tags(tags)

  # log parameters
  mlflow.log_param("epochs", EPOCHS)
  mlflow.log_param("loss_function", str(LOSS))

  # log metrics
  for epoch in range(0, EPOCHS):
    mlflow.log_metric("accuracy", hist['accuracy'][epoch])
    mlflow.log_metric("loss",  hist['loss'][epoch])
    
  mlflow.end_run()

2021/02/04 19:08:19 INFO mlflow.utils.autologging_utils: tensorflow autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow to the MLflow run with ID 'd2c4645e22ea4fd495fbdac6b72a219a'


Epoch 1/500


1/2 [==============>...............] - ETA: 0s - loss: 2.0745 - accuracy: 0.2188

Instructions for updating:
use `tf.profiler.experimental.stop` instead.


2/2 [==============================] - 0s 45ms/step - loss: 2.0743 - accuracy: 0.2121


...

Epoch 499/500
1/2 [==============>...............] - ETA: 0s - loss: 0.6536 - accuracy: 0.9062

2/2 [==============================] - 0s 2ms/step - loss: 0.6575 - accuracy: 0.9091


Epoch 500/500


1/2 [==============>...............] - ETA: 0s - loss: 0.6397 - accuracy: 0.9062

2/2 [==============================] - 0s 2ms/step - loss: 0.6533 - accuracy: 0.9091
